In [45]:
from typing import TypedDict, List, Union
from langgraph.graph import StateGraph, END
import math

In [46]:
class NumberFlowState(TypedDict):
    num1: float
    num2: float
    current_value: Union[float, int, None, str]

In [47]:
graph_builder = StateGraph(NumberFlowState)

In [48]:
def node_a(state: NumberFlowState) -> dict:
    num1 = state.get('num1', 0.0)
    num2 = state.get('num2', 0.0)
    print("Node A: Received inputs")
    return {'num1': num1, 'num2': num2}




In [49]:
def node_b(state: NumberFlowState) -> dict:
    num1 = state['num1']
    num2 = state['num2']
    result = num1 + num2
    print(f"Node B: {num1} + {num2} = {result}")
    return {'current_value': result}

In [50]:
def node_c(state: NumberFlowState) -> dict:
    current_value = state['current_value']
    result = current_value * 5
    print(f"Node C: {current_value} * 5 = {result}")
    return {'current_value': result}

In [51]:
def node_d(state: NumberFlowState) -> dict:
    current_value = state['current_value']
    result = current_value - 10
    print(f"Node D: {current_value} - 10 = {result}")
    return {'current_value': result}

In [52]:
def node_e(state: NumberFlowState) -> dict:
    current_value = state['current_value']
    if current_value is None or (isinstance(current_value, (int, float)) and current_value == 0):
        print("Node E: Division by zero or invalid value, using 0 as current_value")
        current_value = 0.0
    result = current_value / 5.0
    print(f"Node E: {current_value} / 5 = {result}")
    return {'current_value': result}
    

In [53]:
def node_f(state: NumberFlowState) -> dict:
    current_value = state['current_value']
    try:
        int_value = int(round(current_value))
        result = int_value % 7
        print(f"Node F: {int_value} % 7 = {result}")
        return {'current_value': result}
    except (ValueError, TypeError) as e:
         print(f"Node F: Could not perform remainder op on {current_value}: {e}")
         return {'current_value': f"Error: {current_value} not applicable for % 7"}
        


In [54]:
def route_after_e(state: NumberFlowState) -> str:
    current_value = state.get('current_value')
    print(f"Router after E: Checking value type/form: {current_value}")
    if isinstance(current_value, (int, float)):
        is_integer_like = math.isclose(current_value, round(current_value), abs_tol=1e-9)
        print(f"--- Value {current_value} is integer-like: {is_integer_like}")
        if is_integer_like:
             return "is_integer_path" # Route to F
        else:
             return "is_float_path" # Route to END
    else:
        print(f"--- Value {current_value} is not a valid number, routing to float path (END).")
        return "is_float_path" # Treat non-numeric as float path / error path to END




In [55]:
graph_builder.add_node("A", node_a)
graph_builder.add_node("B", node_b)
graph_builder.add_node("C", node_c)
graph_builder.add_node("D", node_d)
graph_builder.add_node("E", node_e)
graph_builder.add_node("F", node_f)

graph_builder.set_entry_point("A")

In [56]:
graph_builder.add_edge("A", "B")
graph_builder.add_edge("B", "C")
graph_builder.add_edge("C", "D")
graph_builder.add_edge("D", "E")
graph_builder.add_conditional_edges("E",route_after_e,{
    "is_integer_path": "F",
    "is_float_path": END
    }
)

graph_builder.add_edge("F", END)


In [57]:
compiled_graph = graph_builder.compile()

In [65]:
compiled_graph.invoke({"num1":10.3, "num2":6.8, })

Node A: Received inputs
Node B: 10.3 + 6.8 = 17.1
Node C: 17.1 * 5 = 85.5
Node D: 85.5 - 10 = 75.5
Node E: 75.5 / 5 = 15.1
Router after E: Checking value type/form: 15.1
--- Value 15.1 is integer-like: False


{'num1': 10.3, 'num2': 6.8, 'current_value': 15.1}

AttributeError: partially initialized module 'yaml' from 'c:\Users\garvi\OneDrive\Desktop\codebases\Gen-AI\venv\Lib\site-packages\yaml\__init__.py' has no attribute 'error' (most likely due to a circular import)